Battery Operations to Utilitize NEM Excess Generation

In [ ]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [ ]:
from datetime import timedelta
import numpy as np

from beo_datastore.libs.battery import Battery, FixedScheduleBatteryIntervalFrame, PeakShavingSimulator

from load.customer.models import Meter

In [ ]:
# Get Meter Data
meter = Meter.objects.first()

# Configure Battery
battery = Battery(rating=5, discharge_duration=timedelta(hours=4), efficiency=0.9, charge=0)

meter.intervalframe.total_frame288.dataframe

In [ ]:
# Generate optimal charge/discharge schedules using exports only
exports_charge_schedule, exports_discharge_schedule = PeakShavingSimulator.optimize_schedules_with_exports(
    battery, meter.intervalframe
)
# Generate optimal charge/discharge schedules using exports and grid
grid_charge_schedule, grid_discharge_schedule = PeakShavingSimulator.optimize_schedules_with_grid(
    battery, meter.intervalframe
)

In [ ]:
# Examine Charge/Discharge Schedules
exports_charge_schedule.dataframe
# exports_discharge_schedule.dataframe
# grid_charge_schedule.dataframe
# grid_discharge_schedule.dataframe

In [ ]:
# Run Battery Operations
exports_intervalframe = FixedScheduleBatteryIntervalFrame(
    battery=battery,
    load_intervalframe=meter.intervalframe,
    charge_schedule=exports_charge_schedule,
    discharge_schedule=exports_discharge_schedule,
)
exports_intervalframe.generate_full_sequence()
grid_intervalframe = FixedScheduleBatteryIntervalFrame(
    battery=battery,
    load_intervalframe=meter.intervalframe,
    charge_schedule=grid_charge_schedule,
    discharge_schedule=grid_discharge_schedule,
)
grid_intervalframe.generate_full_sequence()

In [ ]:
# Examine Impact on Peak Load
exports_intervalframe.compare_peak_loads()
# grid_intervalframe.compare_peak_loads()

In [ ]:
# Examine Impact on Hourly Averages
month = 7
exports_intervalframe.compare_month_hours(7, np.mean)
# grid_intervalframe.compare_month_hours(7, np.mean)

In [ ]:
# Examine Battery Operations per Interval
exports_intervalframe.combined_dataframe
# grid_intervalframe.combined_dataframe